# Script Description
This script executes various CRUD use cases on Azure SQL Database. The time is measured for the various use cases in order to obtain the time required for the respective insert, select, update and delete function. The following use cases are implemented: 

- **Insert one Dataset**: Insert one Dataset as JSON-File in Database
- **Select one dataset in n database records**: Selects based on a single indexed SerialNumber.
- **Select n datasets in n database records via index**: Queries using indexed attributes ArticleName and non-indexed attribute MachineName.
- **Select n datasets in n database records via no-index**: Queries using non-indexed attributes SetpointName and MachineName.
- **Select aggregate in n database records**: Aggregates using ArticleName, MachineName, and InspectionName to compute the average of MeasuredValue.
- **Update one dataset**: Updates a single dataset based on an indexed SerialNumber.
- **Update n datasets via indexed attribute**: Updates datasets using the indexed attribute ArticleName.
- **Update n datasets via non-indexed attribute**: Updates datasets using the non-indexed attribute MachineName.
- **Delete one Dataset**: Delete one Dataset from Azure Cosmos DB

# Importing Libraries

In [ ]:
import pyodbc
import json
from datetime import datetime
import time
import os
import random
from tqdm import tqdm
import csv

# Define Functions

## Helper Functions

Helper functions are utilized to enhance code readability and maintainability. These functions include database connection establishment, saving recorded data to CSV files, and reading SQL files.

The following helper functions are defined:

- **connect_to_database**: Function to establish a connection to the database.
- **read_sql_query_from_file**: Function to read SQL files.
- **store_values_in_csv**: Function to save measured durations of individual use cases into CSV files.
- **get_all_distinct_serialnumbers**: Function to retrieve all distinct serial numbers from the database.
- **get_number_of_processed_datasets**: Function to retrieve the number of datasets to be processed.
- **read_sequence_to_insert_data**: Function to read the file indicating the order of datasets to be created.

In [ ]:
# OK!
def connect_to_database():
    """
    Establishes a connection to the Azure SQL database via a config file.

    Returns:
        connection (pyodbc.Connection): Connection object if successful.
        None: If connection fails.
    """
    
    try:

        config_file_path = 'config.json'

        # Read configuration data from JSON file
        with open(config_file_path, 'r') as file:
            config_data = json.load(file)

        server   = config_data['Server']
        database = config_data['Database']
        username = config_data['Username']
        password = config_data['Password']
        driver   = config_data['Driver']
        
        # Cónnect to Azure SQL Database
        connection = pyodbc.connect(
            f'DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password};'
        )

        print("Successful: Connection to Azure SQL Database successfully established.")

        return connection
    
    except Exception as e:
        print("Error: Error when connecting to Azure SQL Database:", e)
        return None

In [ ]:
# OK!
def read_sql_query_from_file(filename_sql_query):
    """
    Reads the content of a SQL file and returns it as a string.

    Args:
        filename_sql_query (str): The name of the SQL file.

    Returns:
        sql_query (str): The content of the SQL file as a string.
    """
    
    filepath = os.path.join("SQL_Queries", filename_sql_query)

    # Read the content of the sql-file
    with open(filepath, 'r') as file:
        sql_query = file.read()
        
    return sql_query

In [ ]:
# OK!
def store_values_in_csv(dataset, filename_resultfile):
    """
    Appends values to a CSV file or creates a new one if not exist.

    Args:
        dataset (tuple): The data to be written to the CSV file. Each inner list represents a row.
        filename_resultfile (str): The name of the csv-result-file.
        
    Returns:
        None
    """
    
    filepath = os.path.join("Experiment_Results", filename_resultfile)
    
    file_exists = os.path.isfile(filepath)
    
    with open(filepath, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        if not file_exists:
            writer.writerow(['DurationTime', 'NumberOfProcessedDatasets', 'NumberOfDatasetsInDatabase'])
        
        # Append the data in csv file
        writer.writerows(dataset)

In [ ]:
# OK!
def get_all_distinct_serialnumbers():
    """
    Retrieves all unique serial numbers from Azure Cosmos DB.

    Returns:
        serialnumbers (tuple): A tuple containing all unique serial numbers.
    """
    
    try:
        
        
        sql_query = 'SELECT DISTINCT SerialNumber FROM InspectionOperations;'
        
        cursor.execute(sql_query)

        serialnumbers = [row.SerialNumber for row in cursor.fetchall()]
            
        return serialnumbers

    except Exception as e:
        print("An error occurred:", e)
        return None, None

In [ ]:
# OK!
def get_number_of_processed_datasets(filename_sql_query):
    """
    Returns the number of affected datasets based on the serialnumber

    Args:
        filename_sql_query (str): Name of the SQL query file.

    Returns:
        database_record_count (int): Count of affected datasets from the query result.
    """
    
    try:
        
        filepath = os.path.join("SQL_Queries/SQL_Queries_Number_Of_Datasets", filename_sql_query)

        with open(filepath, 'r') as file:
            sql_query = file.read()
        
        cursor.execute(sql_query)

        database_record_count = cursor.fetchone()[0]
        
        return database_record_count
    
    except Exception as e:
        # Print error message and return None if an exception occurs
        print("An error occurred:", e)
        return None


In [ ]:
# OK!
def read_sequence_to_insert_data(sequence_range_start, sequence_range_end):
    """
    Reads a range of data from a CSV file containing sequence of insertion data.

    Args:
        sequence_range_start (int): The starting index of the sequence to read.
        sequence_range_end (int): The ending index of the sequence to read.

    Returns:
        order_numbers (tuple): A tuple containing lists of order numbers.
        serial_numbers (tuple): A tuple containing lists of serial numbers.
        article_names (tuple): A tuple containing lists of article names.
        machine_names (tuple): A tuple containing lists of machine names.
    """
    
    order_numbers  = []
    serial_numbers = []
    article_names  = []
    machine_names  = []

    with open('sequence_of_inserting_data.csv', mode='r') as file:

        reader = csv.reader(file)

        # Skip heading in csv file
        next(reader)

        for _ in range(sequence_range_start - 1):

            # Skip lines up to the start index
            next(reader)

        for _ in range(sequence_range_end - sequence_range_start + 1):
            row = next(reader)
            order_numbers.append(row[0])
            serial_numbers.append(row[1])
            article_names.append(row[2])
            machine_names.append(row[3])
        
        return order_numbers, serial_numbers, article_names, machine_names

In [ ]:
# OK!
def execute_query(sql_query):
    """
    Executes a SQL query and measures the duration of execution.

    Args:
        sql_query (str): The SQL query to be executed.

    Returns:
        query_duration (float): The duration of the query execution in seconds. Returns None if an error occurs.
    """
    try:

        query_start_time = time.time()

        cursor.execute(sql_query)
        
        # In the case of a write operation, the following must be committed
        if "SP_InsertInspectionOperation" in sql_query or "UPDATE" in sql_query or "DELETE" in sql_query:
            connection.commit()

        query_end_time = time.time()
        query_duration = query_end_time - query_start_time
        
        return query_duration
    
    except Exception as e:
        # Print error message and return None if an exception occurs
        print("An error occurred:", e)
        return None


## Query Functions

Query functions serve as the primary operations within the script, encompassing database operations such as Insert, Select, Update, and Delete. The following functions are defined:

- **insert_data**: Prepares and inserts datasets into the database.
- **select_data**: Implements various Select use cases, including aggregate queries.
- **update_data**: Implements various Update use cases.
- **delete_data**: Deletes individual records from the database.

### insert_data
The `insert_data` function is designed to insert datasets into Azure SQL Database. 

It begins by reading a sequence of rows from the file "sequence_of_inserting_data.csv" between the specified `sequence_range_start` and `sequence_range_end`. This file dictates the order in which datasets are to be inserted into the database. Each row in this file contains an order number, serial number, article name, and machine name. This sequence file was generated to ensure both Azure Cosmos DB and Azure SQL Database insert datasets in the same order, facilitating fair performance testing.

_Example of the file "sequence_of_inserting_data.csv":_
| OrderNumber | SerialNumber | ArticleName| MachineName |
|----------|----------|----------|----------|
| 0000001   | 0000001   | Mountainbike   | InspectionMachine3   |
| 0000001   | 0000002   | Mountainbike   | InspectionMachine3   |
| 0000001   | 0000003   | Mountainbike   | InspectionMachine3   |
| 0000001   | 0000004   | Mountainbike   | InspectionMachine3   |
| 0000001   | 0000005   | Mountainbike   | InspectionMachine3   |
| 0000002   | 0000006   | Roadbike   | InspectionMachine1   |
| 0000002   | 0000007   | Roadbike   | InspectionMachine1   |
| 0000002   | 0000008   | Roadbike   | InspectionMachine1   |
| 0000002   | 0000009   | Roadbike   | InspectionMachine1   |
| 0000002  | 0000010  | Roadbike  | InspectionMachine1  |


For each row extracted from "sequence_of_inserting_data.csv," a dataset is generated and inserted into the database. The choice of reference dataset is determined based on the `ArticleName` in the sequence file. The reference dataset is then read, and attributes such as `OrderNumber`, `ArticleName`, `SerialNumber`, and `MachineName` are replaced with values from the extracted row. The `id` attribute is randomly generated using the `uuid` library. The `InsertDateTime` and `UpdateDateTime` attributes are assigned the current timestamp. Measured values for each inspection step are randomly generated based on upper and lower boundary values for each process step.

Upon creating the dataset based on the reference template, it is inserted into the database. The time taken for each insertion operation is measured using the `time` library, and these durations are stored in a list.

Once all datasets from "sequence_of_inserting_data.csv" are inserted, the mean insertion time is calculated and stored in a CSV file.

This function ensures systematic and controlled insertion of datasets into Azure SQL Database, crucial for performance evaluation and benchmarking purposes.

In [ ]:
# OK!
def insert_data(database_record_count, filename_resultfile, sequence_range_start, sequence_range_end, tdqm_description):
    """
    Inserts data into Azure Cosmos DB container and logs the query durations.

    Args:
        database_record_count (int): Total number of datasets in the database.
        filename_resultfile (str): File name for storing query durations as CSV.
        sequence_range_start (int): Starting index of the sequence to read from CSV.
        sequence_range_end (int): Ending index of the sequence to read from CSV.
        tdqm_description (str): Description for tqdm progress bar.

    Returns:
        None

    """
    
    try:
        
        query_durations = []
        
        # Retrieve sequence of ordernumbers, serialnumbers, articlenames and machinenames
        order_numbers, serial_numbers, article_names, machine_names = read_sequence_to_insert_data(sequence_range_start, sequence_range_end)
        
        # Generate the datasets
        for order_number, serial_number, article_name, machine_name in tqdm(zip(order_numbers, serial_numbers, article_names, machine_names), total=len(article_names),desc=tdqm_description):

            reference_dataset_file_path = os.path.join('Reference_Datasets', f"reference_dataset_{article_name}.json")

            with open(reference_dataset_file_path, 'r') as file:
                json_data = json.load(file)

            # Replace data
            json_data['OrderNumber']        = order_number
            json_data['SerialNumber']       = serial_number
            json_data['MachineName']        = machine_name

            # Generate the MeasuredValue for each inspection step
            for inspection in json_data['InspectionsAndResults']:
                lower_border_value = float(inspection['InspectionLowerBorderValue'])
                upper_border_value = float(inspection['InspectionUpperBorderValue'])
                measured_value = round(random.uniform(lower_border_value, upper_border_value), 2)
                inspection['InspectionResultMeasuredValue'] = str(measured_value)
                
            # Convert data to JSON string
            json_data_as_string = json.dumps(json_data)

            try:
                sql_query = "{CALL SP_InsertInspectionOperation('" + json_data_as_string + "')}"
                    
                query_duration = execute_query(sql_query)

                query_durations.append(query_duration)

            except Exception as e:
                print("An error occurred while inserting data:", e)
                continue
                
        mean_duration = sum(query_durations) / len(query_durations)
        
        # Define number of processed datasets to one, because every loop just one dataset is inserted
        number_of_processed_datasets = 1
        dataset_to_store = [[mean_duration, number_of_processed_datasets, database_record_count]] if query_durations else []
        store_values_in_csv(dataset_to_store, filename_resultfile)

    except Exception as e:
        print("An error occurred:", e)
    

### select_data

The `select_data` function executes various SELECT queries on Azure SQL Database to evaluate database performance across different use cases:

- **Select one dataset in n database records**: Selects based on a single indexed SerialNumber.
- **Select n datasets in n database records via index**: Queries using indexed attributes ArticleName and non-indexed attribute MachineName.
- **Select n datasets in n database records via no-index**: Queries using non-indexed attributes SetpointName and MachineName.
- **Select aggregate in n database records**: Aggregates using ArticleName, MachineName, and InspectionName to compute the average of MeasuredValue.

For queries not based on SerialNumber (i.e., multiple datasets), the function first retrieves the count of affected datasets from the database. When querying by SerialNumber (i.e., a single dataset), only one dataset is affected, as each SerialNumber is unique in the database.

During a SerialNumber query, all distinct SerialNumbers are fetched from the database and stored in a list. Then, a random SerialNumber from the list is selected, and the tag "SERIALNUMBER" in the SQL query is replaced with this randomly selected SerialNumber, as illustrated below:

_Before:_  
```sql
SELECT * FROM c WHERE c.SerialNumber = 'SERIALNUMBER'
```

_After:_  
```sql
SELECT * FROM c WHERE c.SerialNumber = '0000042'
```
The adjusted query is then executed.

In [ ]:
# OK!
def select_data(
    number_of_experiments,
    database_record_count,
    filename_resultfile, 
    filename_sql_query, 
    tdqm_description,
    filename_sql_query_count_datasets = None,
    use_serialnumbers = False):
    
    """
    Executes experiments to query data from Azure Cosmos DB and calculates query durations.

    Args:
        number_of_experiments (int): Number of experiment cycles to run.
        database_record_count (int): Total number of datasets in the database.
        filename_resultfile (str): File name for storing mean query durations as CSV.
        filename_sql_query (str): File name containing the SQL query to execute.
        tdqm_description (str): Description for tqdm progress bar.
        filename_sql_query_count_datasets (str, optional): File name containing SQL query to count affected datasets.
        use_serialnumbers (bool, optional): Flag to indicate whether to use distinct serial numbers.

    Returns:
        None

    Raises:
        Prints error messages if query execution fails.

    """

    # Define list to store the query duration times
    query_durations = []

    # Read the sql query from file
    sql_query = read_sql_query_from_file(filename_sql_query)
    
    # Find out how much datasets based on distinc serialnumber are available if the variable is used
    number_of_processed_datasets = get_number_of_processed_datasets(filename_sql_query_count_datasets) if filename_sql_query_count_datasets else 1
        
    # If use_serialnumber is True, get all serial numbers
    serialnumbers = get_all_distinct_serialnumbers() if use_serialnumbers else []
                
    # Run the Experiment
    for experiment_cycle_number in tqdm(range(1, number_of_experiments+1), desc=tdqm_description):
        
        try:
            
            if use_serialnumbers:
                random_serialnumber = random.choice(serialnumbers)
                prepared_sql_query = sql_query.replace('SERIALNUMBER', random_serialnumber)               
            else:
                prepared_sql_query = sql_query
                                
            # Measure query duration
            query_duration = execute_query(prepared_sql_query)

            # Store query duration
            query_durations.append(query_duration)

        except Exception as e:
            print("An error occurred while inserting data:", e)
            continue
            
    # Calculate the mean duration of all duration-times
    mean_duration = sum(query_durations) / len(query_durations)
    
    # Define the dataset to store
    dataset_to_store = [[mean_duration, number_of_processed_datasets, database_record_count]] if query_durations else []
            
    
    # Store values in CSV
    store_values_in_csv(dataset_to_store, filename_resultfile)
            
    
    del query_durations, query_duration, mean_duration, number_of_processed_datasets, database_record_count

### update_data

The `update_data` function is designed to perform various update operations in the database:

- **Update one dataset**: Updates a single dataset based on an indexed SerialNumber.
- **Update n datasets via indexed attribute**: Updates datasets using the indexed attribute ArticleName.
- **Update n datasets via non-indexed attribute**: Updates datasets using the non-indexed attribute MachineName.

In [ ]:
def update_data(
    number_of_experiments,
    database_record_count,
    filename_resultfile, 
    filename_sql_query, 
    tdqm_description,
    filename_sql_query_count_datasets = None,
    use_serialnumbers = False):
    
    """
    Perform data update experiments on Azure SQL Database.

    Args:
        number_of_experiments (int): Number of experiment cycles to run.
        database_record_count (int): Number of database records.
        filename_resultfile (str): Name of the CSV file to store results.
        filename_sql_query (str): Name of the SQL query file.
        tdqm_description (str): Description for tqdm progress bar.
        filename_sql_query_count_datasets (str, optional): File to count datasets.
        use_serialnumbers (bool, optional): Flag to use serial numbers.

    Returns:
        None
    """

    query_durations = []

    # Read the sql query from file
    sql_query = read_sql_query_from_file(filename_sql_query)
    
    # Determine number of datasets based on distinct serial numbers if the query file is provided
    number_of_processed_datasets = get_number_of_processed_datasets(filename_sql_query_count_datasets) if filename_sql_query_count_datasets is not None else 1

    # If use_serialnumbers is True, retrieve all distinct serial numbers
    serialnumbers = get_all_distinct_serialnumbers() if use_serialnumbers else []
    
    # Run the Experiment
    for experiment_cycle_number in tqdm(range(1, number_of_experiments+1), desc=tdqm_description):
        try:
            if use_serialnumbers:
                
                random_serialnumber = random.choice(serialnumbers)
                
                # Replace SQL query with the selected serial number
                prepared_sql_query = sql_query.replace('SERIALNUMBER', random_serialnumber)
                                
            else:
                
                prepared_sql_query = sql_query

            # Execute Query and calculate the time for duration and store it into list
            query_duration = execute_query(prepared_sql_query)
                                
            query_durations.append(query_duration)
            
        except Exception as e:
            print("An error occurred while inserting data:", e)
            continue
    
    
    # Calculate the mean duration of all duration-times
    mean_duration = sum(query_durations) / len(query_durations)
    
    # Define the dataset to store
    dataset_to_store = [[mean_duration, number_of_processed_datasets, database_record_count]] if query_durations else []
    
    # Store values in CSV
    store_values_in_csv(dataset_to_store, filename_resultfile)
            
    del query_durations, query_duration, database_record_count

### DELETE-Queries
This function can be used to delete data from Azure SQL Database based on a serial number. As each serial number only occurs once in the database, exactly one data record is deleted based on the described use case. 

After all database operations have been carried out on 100,000 data records, all data is deleted from the database at once. Then 10,000 new data records are inserted. All serial numbers are then retrieved from the database and saved in a list. A random serial number is then selected from the list and the corresponding data record is deleted from the database. As soon as the data record has been successfully deleted from the database, the serial number is also removed from the list. The process is repeated until all serial numbers have been deleted from the list. In this case, all data in the database is also deleted.

If this attempt were to be carried out on 100,000 data records, it would take 13.7 days. For this reason, the test is only carried out on 10,000 data records.

In [ ]:
def delete_data(filename_sql_query, filename_resultfile):
    """
    Deletes data from Azure Cosmos DB based on serial numbers.

    Args:
    - filename_resultfile (str): The filename to store query durations and metrics in CSV format.
    """
    
    query_durations = []
    
    
    serialnumbers = get_all_distinct_serialnumbers()
    
    # Read the sql query from file
    sql_query = read_sql_query_from_file(filename_sql_query)
    
    number_of_all_datasets = len(serialnumbers)  # 10.000    
    number_of_datasets_to_delete_per_iteration = number_of_all_datasets // 10  # 1.000
    
    with tqdm(total=len(serialnumbers), desc="Deleting datasets from database") as progressbar:
        
        # 10, because want to have 10 datapoints!
        for _ in range(10):
            iteration_durations = []
            
            for _ in range(number_of_datasets_to_delete_per_iteration):
                
                if not serialnumbers:
                    break
                
                try:
                    
                    random_serialnumber = random.choice(serialnumbers)

                    prepared_sql_query = sql_query.replace('SERIALNUMBER', random_serialnumber)

                    query_duration = execute_query(prepared_sql_query)

                    iteration_durations.append(query_duration)

                    serialnumbers.remove(random_serialnumber)

                    # Update progress bar
                    progressbar.update(1)
                    
                
                except Exception as e:
                    print("An error occurred:", e)
            
            if iteration_durations:
                mean_duration = sum(iteration_durations) / len(iteration_durations)
                
                query_durations.append([mean_duration, 1, number_of_all_datasets])
            
                number_of_all_datasets = number_of_all_datasets - number_of_datasets_to_delete_per_iteration
    
    store_values_in_csv(query_durations, filename_resultfile)
    
    del query_durations


## Experiment-Excecution Function

In [ ]:
def run_experiment(database_record_count, sequence_range_start, sequence_range_end):
    
    print(f'START Experiment with {database_record_count} Datasets!')

    # Use Case 1 - Insert datasets
    insert_data(
        database_record_count = database_record_count,
        filename_resultfile = "uc1_result_file.csv",
        sequence_range_start = sequence_range_start,
        sequence_range_end = sequence_range_end,
        tdqm_description = f'INSERT {database_record_count} Datasets'
    )

    # Use Case 2 - Select one datasets
    select_data(
        database_record_count = database_record_count,
        number_of_experiments = 10, 
        filename_resultfile = "uc2_result_file.csv", 
        filename_sql_query = "uc2_sql_query.sql", 
        tdqm_description = f'SELECT {database_record_count} Datasets (Serialnumber)', 
        use_serialnumbers=True
    )

    # Use Case 3 - Select n datasets based on index
    select_data(
        database_record_count = database_record_count,
        number_of_experiments =3, 
        filename_resultfile ='uc3_result_file.csv', 
        filename_sql_query ="uc3_sql_query.sql", 
        filename_sql_query_count_datasets ="uc3_sql_query.sql",
        tdqm_description = f'SELECT {database_record_count} Datasets (Article Names)',
        use_serialnumbers =False
    )

    # Use Case 4 - Select n datasets based on non-index
    select_data(
        database_record_count = database_record_count,
        number_of_experiments=3, 
        filename_resultfile='uc4_result_file.csv', 
        filename_sql_query="uc4_sql_query.sql", 
        filename_sql_query_count_datasets="uc4_sql_query.sql",
        tdqm_description = f'SELECT {database_record_count} Datasets (Setpoints)',
        use_serialnumbers=False
    )

    # Use Case 5 - Select aggregate
    select_data(
        database_record_count = database_record_count,
        number_of_experiments=3, 
        filename_resultfile='uc5_result_file.csv', 
        filename_sql_query="uc5_sql_query.sql", 
        filename_sql_query_count_datasets="uc5_sql_query.sql", 
        use_serialnumbers=False,
        tdqm_description = f'SELECT AGGREGATE {database_record_count} Datasets (Article Name)'
    )
    
    # Use Case 6 - Update one dataset
    update_data(
        database_record_count = database_record_count,
        number_of_experiments=10,
        filename_resultfile='uc6_result_file.csv', 
        filename_sql_query="uc6_sql_query.sql", 
        filename_sql_query_count_datasets=None, 
        use_serialnumbers=True,
        tdqm_description = f'UPDATE {database_record_count} Datasets (Serialnumber)'
    )

    # Use Case 7 - Update n datasets based on index
    update_data(
        database_record_count = database_record_count,
        number_of_experiments=3,
        filename_resultfile='uc7_result_file.csv', 
        filename_sql_query="uc7_sql_query.sql", 
        tdqm_description = f'UPDATE {database_record_count} Datasets (Articlename)',
        filename_sql_query_count_datasets="uc7_sql_query.sql", 
        use_serialnumbers=False,
    )
    
    # Use Case 8 - Update n datasets based on non-index
    update_data(
        database_record_count = database_record_count,
        number_of_experiments=3,
        filename_resultfile='uc8_result_file.csv', 
        filename_sql_query="uc8_sql_query.sql", 
        tdqm_description = f'UPDATE {database_record_count} Datasets (Machinename)',
        filename_sql_query_count_datasets="uc8_sql_query.sql", 
        use_serialnumbers=False,
    )
    
    print(f'END Experiment with {database_record_count} Datasets!')

# Main-Script

In [ ]:
experiment_start_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f'Experiment Start-Time: {experiment_start_time}')

connection = connect_to_database()

for i in range(10):
    # Calculation of sequence_range_start and sequence_range_end based on the index of the loop
    sequence_range_start = (i * 10000) + 1
    sequence_range_end = (i + 1) * 10000
    
    database_record_count = sequence_range_end
    
    cursor = connection.cursor()
    
    run_experiment(database_record_count, sequence_range_start, sequence_range_end)
    
    cursor.close()



# Use Case 9 - Delete datasets

cursor = connection.cursor()

cursor.execute("EXEC SP_DeleteAllInspectionData")
connection.commit()

insert_data(
    database_record_count = 10000,
    filename_resultfile = "dummy_insert.csv",
    sequence_range_start = 1,
    sequence_range_end = 10000,
    tdqm_description = f'INSERT 10000 Datasets'
    )


delete_data(
    filename_sql_query = "uc9_sql_query.sql", 
    filename_resultfile = "uc9_result_file.csv"
)

connection.close()

experiment_end_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f'Experiment End-Time: {experiment_end_time}')